### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = tomorrow

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
# ligas = temporada[temporada['Date'] == data_var]['League'].unique().tolist()
# temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos.to_csv(f'jogos_processados/{data_var}.csv', index=False)

jogos[['Date', 'League', 'Time', 'Home', 'Away', 
       'Odds_H', 'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', ]]

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A
6163,2023-11-11,Brasil Nbb,16:00,Brasília,Caxias do Sul,1.61,2.28,154.5,1.86,1.88,-4.5,2.01,1.72
6164,2023-11-11,Brasil Nbb,17:00,Bauru,Pinheiros,1.42,2.81,154.5,1.89,1.87,-6.5,1.99,1.73
6165,2023-11-11,Espanha Acb,14:00,Breogan,Joventut,1.79,2.07,157.5,1.86,1.92,-2.5,2.03,1.77
6166,2023-11-11,Espanha Acb,14:00,MoraBanc Andorra,Manresa,1.67,2.27,169.5,1.81,1.94,-4.5,2.09,1.69
6167,2023-11-11,Espanha Acb,16:45,Basquet Girona,Unicaja,2.86,1.45,166.5,1.85,1.91,4.5,1.98,1.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6405,2023-11-11,Áustria Superliga,15:00,Kapfenberg,Furstenfeld,1.01,13.70,162.5,1.86,1.80,-26.5,1.99,1.73
6406,2023-11-11,Chile Copa Do Chile,19:45,Liceo Pablo Neruda,Leones Quilpue,11.60,1.02,154.5,1.80,1.87,33.5,2.00,1.72
6407,2023-11-11,Finlândia I Divisão A,13:00,Honka,Oulu,1.17,4.57,158.5,1.83,1.83,-14.5,1.86,1.80
6408,2023-11-11,Grécia Liga De Elite,14:00,Mykonos,Lefkadas,1.17,4.48,155.5,1.80,1.86,-10.5,1.82,1.84


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
6,00:00,Eua Nba,Phoenix Suns,Los Angeles Lakers,223.5,1.86,1.0000,Over
7,00:00,Eua Nba,Sacramento Kings,Oklahoma City Thunder,230.5,1.86,1.0000,Over
8,21:30,Eua Nba,Atlanta Hawks,Miami Heat,228.5,1.97,0.5269,Over


### Handicap positivo V4 (LDA)

In [9]:
# Duplicando o dataset
df_hp_v4 = jogos.copy()

df_hp_v4.loc[(df_hp_v4['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_hp_v4.loc[(df_hp_v4['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_hp_v4['Zebra'] = df_hp_v4['Zebra'].astype('category')

# Odds
df_hp_v4.loc[(df_hp_v4['Zebra']) == 'Casa', 'Odds'] = df_hp_v4['HA_Odds_H']
df_hp_v4.loc[(df_hp_v4['Zebra']) == 'Fora', 'Odds'] = df_hp_v4['HA_Odds_A']

df_hp_v4.reset_index(drop=True, inplace=True)
df_hp_v4.replace([np.inf, -np.inf], 0, inplace=True)

if not df_hp_v4.empty:
    mdl_hp_v4 = load_model('ML/handicap_positivo/v4/v4_handicap_positivo', verbose=False)
    prev = predict_model(mdl_hp_v4, df_hp_v4)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Over_Line'] > 155))]

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/handicap_positivo_v4_{data_var}.csv', index=False)

        print('------------ MÉTODO HANDICAP POSITIVO V4 (LDA) ------------')
        print()

        prev.loc[prev['Zebra'] == 'Casa', 'Bet'] = 'HA Casa'
        prev.loc[prev['Zebra'] == 'Fora', 'Bet'] = 'HA Fora'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO HANDICAP POSITIVO V4 (LDA) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
4,16:45,Espanha Acb,Basquet Girona,Unicaja,4.5,1.98,0.6793,HA Casa
8,21:30,Eua Nba,Atlanta Hawks,Miami Heat,-6.5,1.90,0.9998,HA Fora
10,15:30,Itália Liga A,Virtus Bologna,Treviso,-17.5,1.72,0.6179,HA Fora
11,16:30,Itália Liga A,Basket Napoli,Varese,-9.5,1.66,0.8757,HA Fora
14,16:00,Alemanha Bbl,Heidelberg,Wurzburg,-2.5,1.74,0.9866,HA Fora
18,21:00,Argentina Liga A,Union De Santa Fe,Obras,3.5,2.02,0.9995,HA Casa
21,06:00,Austrália Nbl,Adelaide 36ers,Tasmania JackJumpers,2.5,1.99,0.9989,HA Casa
23,07:30,Austrália Liga Feminina Wnbl,Perth Lynx F,Adelaide Lightning F,-7.5,1.66,0.9913,HA Fora
31,13:00,Bósnia E Herzegovina Prvenstvo Bih,Prijedor,Orlovik,5.5,2.00,0.5030,HA Casa
33,13:00,Bulgária Nbl,Cherno More,Chernomorets,16.5,1.97,0.6665,HA Casa


### Handicap positivo V5 (RF)

In [10]:
# Duplicando o dataset
df_hp_v5 = jogos.copy()

df_hp_v5.loc[(df_hp_v5['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_hp_v5.loc[(df_hp_v5['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_hp_v5['Zebra'] = df_hp_v5['Zebra'].astype('category')

# Odds
df_hp_v5.loc[(df_hp_v5['Zebra']) == 'Casa', 'Odds'] = df_hp_v5['HA_Odds_H']
df_hp_v5.loc[(df_hp_v5['Zebra']) == 'Fora', 'Odds'] = df_hp_v5['HA_Odds_A']

df_hp_v5.reset_index(drop=True, inplace=True)
df_hp_v5.replace([np.inf, -np.inf], 0, inplace=True)

if not df_hp_v5.empty:
    mdl_hp_v5 = load_model('ML/handicap_positivo/v5/v5_handicap_positivo', verbose=False)
    prev = predict_model(mdl_hp_v5, df_hp_v5)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Over_Line'] > 155))]

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/handicap_positivo_v5_{data_var}.csv', index=False)

        print('------------ MÉTODO HANDICAP POSITIVO V5 (RF) ------------')
        print()

        prev.loc[prev['Zebra'] == 'Casa', 'Bet'] = 'HA Casa'
        prev.loc[prev['Zebra'] == 'Fora', 'Bet'] = 'HA Fora'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO HANDICAP POSITIVO V5 (RF) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
4,16:45,Espanha Acb,Basquet Girona,Unicaja,4.5,1.98,0.5200,HA Casa
8,21:30,Eua Nba,Atlanta Hawks,Miami Heat,-6.5,1.90,0.5300,HA Fora
10,15:30,Itália Liga A,Virtus Bologna,Treviso,-17.5,1.72,0.5400,HA Fora
11,16:30,Itália Liga A,Basket Napoli,Varese,-9.5,1.66,0.6100,HA Fora
14,16:00,Alemanha Bbl,Heidelberg,Wurzburg,-2.5,1.74,0.6300,HA Fora
...,...,...,...,...,...,...,...,...
228,12:00,Turquia Superliga,Anadolu Efes,Tofas,-13.5,1.72,0.5500,HA Fora
229,14:30,Turquia Superliga,Fenerbahce,Merkezefendi,-21.5,1.69,0.5500,HA Fora
231,10:30,Turquia Tbl,Ankaragucu Mamak,Ankara Kolejliler,1.5,1.84,0.5233,HA Casa
244,13:00,Finlândia I Divisão A,Honka,Oulu,-14.5,1.80,0.6100,HA Fora


### Handicap positivo V6 (LDA)

In [11]:
# Duplicando o dataset
df_hp_v6 = jogos.copy()

df_hp_v6.loc[(df_hp_v6['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_hp_v6.loc[(df_hp_v6['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_hp_v6['Zebra'] = df_hp_v6['Zebra'].astype('category')

# Odds
df_hp_v6.loc[(df_hp_v6['Zebra']) == 'Casa', 'Odds'] = df_hp_v6['HA_Odds_H']
df_hp_v6.loc[(df_hp_v6['Zebra']) == 'Fora', 'Odds'] = df_hp_v6['HA_Odds_A']

df_hp_v6.reset_index(drop=True, inplace=True)
df_hp_v6.replace([np.inf, -np.inf], 0, inplace=True)

if not df_hp_v6.empty:
    mdl_hp_v6 = load_model('ML/handicap_positivo/v5/v5_handicap_positivo', verbose=False)
    prev = predict_model(mdl_hp_v6, df_hp_v6)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/handicap_positivo_v6_{data_var}.csv', index=False)

        print('------------ MÉTODO HANDICAP POSITIVO V6 (LDA) ------------')
        print()

        prev.loc[prev['Zebra'] == 'Casa', 'Bet'] = 'HA Casa'
        prev.loc[prev['Zebra'] == 'Fora', 'Bet'] = 'HA Fora'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO HANDICAP POSITIVO V6 (LDA) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
0,16:00,Brasil Nbb,Brasília,Caxias do Sul,-4.5,1.72,0.6000,HA Fora
1,17:00,Brasil Nbb,Bauru,Pinheiros,-6.5,1.73,0.6000,HA Fora
4,16:45,Espanha Acb,Basquet Girona,Unicaja,4.5,1.98,0.5200,HA Casa
8,21:30,Eua Nba,Atlanta Hawks,Miami Heat,-6.5,1.90,0.5300,HA Fora
10,15:30,Itália Liga A,Virtus Bologna,Treviso,-17.5,1.72,0.5400,HA Fora
...,...,...,...,...,...,...,...,...
238,12:00,Turquia Tb2L,Akhisar Belediye,Tesvikiye,-8.5,1.76,0.8300,HA Fora
239,12:00,Turquia Tb2L,Bersa,Cozum Ankara,2.5,1.83,0.5883,HA Casa
240,12:00,Turquia Tb2L,Degerleme,Ege Universitesi,-1.5,1.83,0.5417,HA Fora
244,13:00,Finlândia I Divisão A,Honka,Oulu,-14.5,1.80,0.6100,HA Fora
